In [2]:
from keras import backend as K
import numpy as np
from keras.layers import Input


def square(x):
    return K.square(x)

a = np.array([3.0, 2.0])
print(K.eval(square(K.constant(a))))

[9. 4.]


Using TensorFlow backend.


# Good Resource:

http://laid.delanover.com/gradients-in-tensorflow/

In [144]:
i = K.placeholder(shape=(2,), name="input")
s = square(i)
grads = K.gradients(s, i)[0]
print(grads)
# iterate = K.function([input_img], [loss, grads])
f = K.function([i], [grads])
print(f([a]))

Tensor("gradients_96/Square_50_grad/Mul_1:0", shape=(2,), dtype=float32)
[array([6., 4.], dtype=float32)]


In [142]:
from keras import backend as K
m = K.placeholder(shape=(2,), name="input")
square_f = K.square(m)
grad = K.gradients([square_f], [m])[0]
f = K.function([m], [square_f, grad])
ival = np.array([3.0, 2.0])
print(f([ival]))

[array([9., 4.], dtype=float32), array([6., 4.], dtype=float32)]


In [146]:
import scipy.optimize as so
def f(x):
    return (x[0]*x[1]-1)**2+1, [(x[0]*x[1]-1)*x[1], (x[0]*x[1]-1)*x[0]]
g = np.array([0.1,0.1])
b=[(-10,10),(-10,10)]
so.fmin_tnc(f,g,bounds=b)

(array([1., 1.]), 17, 1)

In [65]:
import scipy.optimize as so

def f(x):   # The rosenbrock function
    return .5*(1 - x[0])**2 + (x[1] - x[0]**2)**2

# Derivative of f
def fprime(x):
    return np.array((-2*.5*(1 - x[0]) - 4*x[0]*(x[1] - x[0]**2), 2*(x[1] - x[0]**2)))

so.fmin_l_bfgs_b(f, [2, 2], fprime=fprime)

(array([1.00000005, 1.00000009]),
 1.441767747301186e-15,
 {'grad': array([ 1.02331202e-07, -2.59299369e-08]),
  'task': b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL',
  'funcalls': 17,
  'nit': 16,
  'warnflag': 0})

## Tensorflow 2.0

In [3]:
import tensorflow as tf

x = tf.placeholder(shape=(2,), dtype = tf.float64)

def f(x):
    return x*x

def grad(x):
    with tf.GradientTape() as t:
        t.watch(x)
        out = f(x)
    return t.gradient(out, x)

a = np.array([3.0, 2.0])

# tf 2.0
# x = tf.convert_to_tensor(a)
# grad(x).numpy()

# tf 1.x
sess = tf.Session()
print(sess.run(grad(x), feed_dict={x: a}))

[6. 4.]


In [13]:
x = tf.ones((2, 2))

with tf.GradientTape() as t:
    t.watch(x)
    y = tf.reduce_sum(x)
    z = tf.multiply(y, y)

# Use the tape to compute the derivative of z with respect to the
# intermediate value y.
dz_dy = t.gradient(z, y)
sess = tf.Session()
print(sess.run(dz_dy))
# assert dz_dy.numpy() == 8.0

8.0


In [172]:
x = tf.ones((2, 2))

with tf.GradientTape() as t:
    t.watch(x)
    y = tf.reduce_sum(x)
    z = tf.multiply(y, y)

# Use the tape to compute the derivative of z with respect to the
# intermediate value y.
dz_dx = t.gradient(z, x)
sess = tf.Session()
print(sess.run(dz_dx))

[[8. 8.]
 [8. 8.]]


## This is what I like :)

In [5]:
import numpy as np
import tensorflow as tf

x =  tf.placeholder(dtype=tf.float32, shape=(2,))

def f(x):   # The rosenbrock function
    return .5*(1 - x[0])**2 + (x[1] - x[0]**2)**2
#     return .5*(1 - tf.gather(x, 0))**2 + (tf.gather(x, 1) - tf.gather(x, 0)**2)**2

# Derivative of f
def fprime(x):
    with tf.GradientTape() as t:
        t.watch(x)
        out = f(x)
    return t.gradient(out, x)

a = np.array([2.0, 2.0])
grad = fprime(x)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for _ in range(2000):
        grad_evaluated = sess.run(grad, feed_dict={x: a})
        a = a - 0.1*grad_evaluated
print(a)
#     optimizer.apply_gradients([(grads, init_image)])

[1.00000006 1.00000004]


In [49]:
import numpy as np
import tensorflow as tf
# import tensorflow.contrib.eager as tfe
# tfe.enable_eager_execution()

# x = tf.constant([2.0, 3.0])
x = tf.Variable([2.0, 3.0], dtype=tf.float32)


def f(x):   # The rosenbrock function
    return .5*(1 - x[0])**2 + (x[1] - x[0]**2)**2
#     return .5*(1 - tf.gather(x, 0))**2 + (tf.gather(x, 1) - tf.gather(x, 0)**2)**2


# Derivative of f
def fprime(x):
    with tf.GradientTape() as t:
        t.watch(x)
        out = f(x)
    return t.gradient(out, x)


a = np.array([2.0, 2.0])
# sess = tf.Session()
# print(sess.run(fprime(x), feed_dict={x: a}))

# optimizer = tf.train.AdamOptimizer()
optimizer = tf.train.GradientDescentOptimizer(0.5)

for i in range(10):
#     print("Iteration: {}".format(i))
#     grads = sess.run(fprime(x), feed_dict={x: a})
    grads = fprime(x)
    updates = optimizer.apply_gradients([(grads, x)])
    print(updates)

None
None
None
None
None
None
None
None
None
None


In [73]:
import numpy as np
import tensorflow as tf
# import tensorflow.contrib.eager as tfe
# tfe.enable_eager_execution()

# x = tf.constant([2.0, 3.0])
x = tf.Variable([2.0, 3.0], dtype=tf.float32)


def f(x):   # The rosenbrock function
    return .5*(1 - x[0])**2 + (x[1] - x[0]**2)**2
#     return .5*(1 - tf.gather(x, 0))**2 + (tf.gather(x, 1) - tf.gather(x, 0)**2)**2


# Derivative of f
def fprime(x):
    with tf.GradientTape() as t:
        t.watch(x)
        out = f(x)
    return t.gradient(out, x)


a = np.array([2.0, 2.0])

# print(sess.run(fprime(x), feed_dict={x: a}))

# optimizer = tf.train.AdamOptimizer()
optimizer = tf.train.GradientDescentOptimizer(0.5)
grads = fprime(x)
trainables = tf.trainable_variables()
updates = optimizer.apply_gradients(zip(grads, trainables))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(10):
        print(sess.run(updates, feed_dict={x: a}))

ValueError: No variables provided.

In [66]:
import numpy as np
import tensorflow as tf
# import tensorflow.contrib.eager as tfe
# tfe.enable_eager_execution()


x = tf.Variable([2.0, 2.0], dtype=tf.float32)


# def f(x):   # The rosenbrock function
#     return .5*(1 - x[0])**2 + (x[1] - x[0]**2)**2
#     return .5*(1 - tf.gather(x, 0))**2 + (tf.gather(x, 1) - tf.gather(x, 0)**2)**2


# Derivative of f
# def fprime(x):
#     with tf.GradientTape() as t:
#         t.watch(x)
#         out = f(x)
#     return t.gradient(out, x)


# a = np.array([2.0, 2.0])

f = .5*(1 - x[0])**2 + (x[1] - x[0]**2)**2

optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(f)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Start training
with tf.Session() as sess:
    # Run the initializer
    sess.run(init)
    for _ in range(100):
        sess.run([optimizer])
    print(sess.run(x))

[1.0509328 1.1144927]


In [1]:
import numpy as np
import tensorflow as tf
# import tensorflow.contrib.eager as tfe
# tfe.enable_eager_execution()


x = tf.Variable([2.0, 2.0], dtype=tf.float32)


# def f(x):   # The rosenbrock function
#     return .5*(1 - x[0])**2 + (x[1] - x[0]**2)**2


# # Derivative of f
# def fprime(x):
#     with tf.GradientTape() as t:
#         t.watch(x)
#         out = f(x)
#     return t.gradient(out, x)

f = .5*(1 - x[0])**2 + (x[1] - x[0]**2)**2
optimizer = tf.train.GradientDescentOptimizer(0.1)
grads = optimizer.compute_gradients(f, var_list=tf.trainable_variables())
train_step = optimizer.apply_gradients(grads)
# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Start training
with tf.Session() as sess:
    # Run the initializer
    sess.run(init)
    for _ in range(100):
        sess.run([train_step])
    print(sess.run(x))

/Users/miladtoutounchian/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


[1.0509328 1.1144927]


In [62]:
import tensorflow as tf

x = tf.Variable(10.0, trainable=True)
f_x = 2 * x* x - 5 *x + 4

loss = f_x
opt = tf.train.GradientDescentOptimizer(0.1).minimize(f_x)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(100):
        print(sess.run([x,loss]))
        sess.run(opt)

[10.0, 154.0]
[6.5, 56.0]
[4.3999996, 20.719995]
[3.1399999, 8.019199]
[2.3839998, 3.4469109]
[1.9303999, 1.8008881]
[1.65824, 1.2083197]
[1.494944, 0.9949951]
[1.3969663, 0.9181981]
[1.3381798, 0.89055157]
[1.302908, 0.88059855]
[1.2817447, 0.8770151]
[1.2690468, 0.8757255]
[1.2614281, 0.87526155]
[1.2568569, 0.87509394]
[1.2541142, 0.87503386]
[1.2524685, 0.87501216]
[1.251481, 0.8750043]
[1.2508886, 0.87500143]
[1.2505331, 0.8750005]
[1.2503198, 0.875]
[1.2501919, 0.87500024]
[1.2501152, 0.87499976]
[1.2500691, 0.875]
[1.2500415, 0.875]
[1.2500249, 0.87500024]
[1.2500149, 0.87500024]
[1.250009, 0.875]
[1.2500054, 0.87500024]
[1.2500032, 0.875]
[1.2500019, 0.875]
[1.2500012, 0.87500024]
[1.2500007, 0.87499976]
[1.2500005, 0.875]
[1.2500002, 0.87500024]
[1.2500001, 0.87500024]
[1.2500001, 0.87500024]
[1.2500001, 0.87500024]
[1.2500001, 0.87500024]
[1.2500001, 0.87500024]
[1.2500001, 0.87500024]
[1.2500001, 0.87500024]
[1.2500001, 0.87500024]
[1.2500001, 0.87500024]
[1.2500001, 0.87500